In [1]:
import pandas as pd
import numpy as np

from datetime import timedelta
import datetime

#import custom functions
import mass_mobil_functions as mm


# code to set option to display all columns taken from this article: https://towardsdatascience.com/how-to-show-all-columns-rows-of-a-pandas-dataframe-c49d4507fcf
pd.set_option('display.max_columns', None)

# <a name="top"></a> Mass Mobilizations and World Changing:
## A Study of the Mass Mobilization Dataset Alongside World Governance Metrics


### Data Cleaning and Engineering


### Project Contents:
|Name|Description|
|---|---|
|1_EDA_Cleaning|Cleans and merges data from 3 sources. Initial EDA.|
|2_Case_Studies|A closer look at a few countries.|
|3_Classification Model|Builds various classification models.|
|4_Visualizations|A visual exploration of the data.|
|data|folder containing csv files of data used|
|streamlit|folder containing streamlit.py file, folder with streamlit images, folder with streamlit data files|


You are currently in notebook 1_EDA_Cleaning. 



#### Notebook Table of Contents

* [Cleaning the Mass Mobilization Data](#clean_mm)
* [Cleaning the Freedom in the World Data](#clean_fiw)
* [Cleaning the WorldBank DataBank Data](#clean_wb)
* [Merging and export the data](#merge)

#### Notebook summary: 
Preparation for exploring this data involved considerable trial and error. I was particularly interested in bringing in various metrics from the World Bank's Databank, particularly those related to World Governance Indicators, GDP, and Poverty and Equity Indicators. After careful consideration, I felt obligated to drop use of the GDP and Poverty and Equity indicators. The data included too many nulls and would have led to considerable loss of data across merges with the Mass Mobilization and World Governance Indicator datasets. I do feel that this is an area for further focus. With more time, I would perform closer analysis and cleaning of the Poverty and Equity indicators, as well as the available GDP data, so that the data may become more useful. 

For this study, I am limiting my data to the years 2006-2021, since these are the years that align across all 3 data sources. I may consider breaking into early data in other capacities in the future. 

I will also take this moment to note that unfortunately the Mass Mobilization dataset does not include data for the United States or Australia.

In [2]:
#reading in mass mobilization dataset
mm = pd.read_csv('../data/raw/dataverse_files/mmALL_073120_csv.csv')
#reading in freedom in the world dataset
fiw = pd.read_csv('../data/transformed/fiw_agg.csv')
#reading in freedom in the world dataset
wgi = pd.read_csv('../data/raw/Data_Extract_From_Worldwide_Governance_Indicators/3f2d17f2-cc70-48f7-8fb9-ab3d88c6b0bd_Data.csv')


In [3]:
# data for protests in the United States are not included in the Mass Mobilization dataset. 

mm[mm['region'] == 'North America']['country'].unique()

array(['Canada', 'Cuba', 'Haiti', 'Dominican Republic', 'Jamaica',
       'Mexico'], dtype=object)

#### Comparing country names in World Governance Indicators (wgi) and Mass Mobilization (mm) 

The following countries are not represented the same in wgi, and I will check for spelling discrepancies:
* 'Venezuela', 'Macedonia', 'Bosnia', 'Yugoslavia',
       'Serbia and Montenegro', 'Russia', 'Cape Verde', 'Gambia',
       'Ivory Coast', 'Congo Brazzaville', 'Congo Kinshasa', 'Swaziland',
       'Iran', 'Egypt', 'Syria', 'Yemen', 'United Arab Emirate',
       'Kyrgyzstan', 'Taiwan', 'North Korea', 'South Korea',
       'Timor Leste'
       * rename wgi 'Venezuela, RB' to 'Venezuela'
       * rename mm Macedonia to North Macedonia
       * rename mm Bosnia to 'Bosnia and Herzegovina'
       * rename mm Yugoslavia to Serbia
       * rename mm Serbia and Montenegro to ??? Serbia? Montenegro? 
       * rename mm Russia to Russian Federation
       * rename mm cape verde to Cabo Verde
       * rename mm Gambia to 'Gambia, The'
       * rename mm Ivory Coast to  'Cote d'Ivoire'
       * rename mm Congo Brazzaville to 'Congo, Rep'
       * rename mm Congo Kinshasa to 'Congo, Dem. Rep.'
       * rename mm Swaziland to 'Eswatini'
       * rename wgi 'Iran, Islamic Rep.' to 'Iran'
       * rename wgi 'Egypt, Arab Rep.' to 'Egypt'
       * rename wgi Syrian Arab Republic to 'Syria'
       * rename wgi 'Yemen, Rep.' to 'Yemen'
       * rename mm 'United Arab Emirate' to 'United Arab Emirates'
       * rename wgi 'Kyrgyz Republic' to 'Kyrgyzstan'
       * rename wgi 'Taiwan, China' to 'Taiwan'
       * rename wgi "Korea, Dem. People's Rep." to 'North Korea'
       * rename wgi 'Korea, Rep.' to 'South Korea'
       * rename mm 'Timor Leste' to 'Timor-Leste'

In [4]:
mm.head(3)

,id,country,ccode,year,region,protest,protestnumber,startday,startmonth,startyear,endday,endmonth,endyear,protesterviolence,location,participants_category,participants,protesteridentity,protesterdemand1,protesterdemand2,protesterdemand3,protesterdemand4,stateresponse1,stateresponse2,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes
0,201990001,Canada,20,1990,North America,1,1,15.0,1.0,1990.0,15.0,1.0,1990.0,0.0,national,NaN,1000s,unspecified,"political behavior, process",labor wage dispute,NaN,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...
1,201990002,Canada,20,1990,North America,1,2,25.0,6.0,1990.0,25.0,6.0,1990.0,0.0,"Montreal, Quebec",NaN,1000,unspecified,"political behavior, process",NaN,NaN,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...
2,201990003,Canada,20,1990,North America,1,3,1.0,7.0,1990.0,1.0,7.0,1990.0,0.0,"Montreal, Quebec",NaN,500,separatist parti quebecois,"political behavior, process",NaN,NaN,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. quebec protest after queen calls for unity ...,"the queen, after calling on canadians to remai..."


In [5]:
wgi.head(3)

,Country Name,Country Code,Series Name,Series Code,1996 [YR1996],1998 [YR1998],2000 [YR2000],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Afghanistan,AFG,Control of Corruption: Estimate,CC.EST,-1.291705,-1.180848,-1.29538,-1.263366,-1.351042,-1.345281,-1.443609,-1.43289,-1.587331,-1.638287,-1.534796,-1.636177,-1.579174,-1.419741,-1.43651,-1.354829,-1.342216,-1.526172,-1.515626,-1.487624,-1.400733,-1.475405
1,Afghanistan,AFG,Control of Corruption: Number of Sources,CC.NO.SRC,2,2,2,2,3,5,5,7,6,7,8,9,9,10,11,11,11,10,10,10,10,9
2,Afghanistan,AFG,Control of Corruption: Percentile Rank,CC.PER.RNK,4.301075,9.793815,5.076142,5.050505,5.050505,5.853659,2.439024,3.902439,0.9708738,0.4854369,0.9569378,0.952381,0.9478673,2.369668,1.895735,5.288462,6.25,3.365385,3.846154,4.807693,6.730769,5.288462


In [6]:
# comparing country names in wgi and mm. 

missing_countries_not_in_mm = list(mm[~mm['country'].isin(wgi['Country Name'])]['country'].unique())
missing_countries_not_in_mm[:5]

['Venezuela', 'Germany West', 'Germany East', 'Czechoslovakia', 'Macedonia']

In [7]:
missing_countries_not_in_wgi = list(wgi[~wgi['Country Name'].isin(mm['country'])]['Country Name'].unique())
missing_countries_not_in_wgi[:5]

['American Samoa', 'Andorra', 'Anguilla', 'Antigua and Barbuda', 'Aruba']

### <a name="clean_mm"></a>Cleaning Mass Mobilization Data
[Jump to the top of this notebook](#top)

#### Null Handling

The Mass Mobilization dataset contained a considerable number of nulls. After careful examination of the data, I handled the nulls in the following ways:
* startday/startmonth/startyear/endday/endmonth/endyear:  drop nulls.
* protestor violence:  If null, I assumed nonviolence (indicated by a 0)
* location: this column will be dropped, as I am not currently looking into granular location, but am focusing on countries as a whole. 
* participants_category: This column was a late addition to the Mass Mobilization dataset, as indicated in the documentation for the data. The researchers initially used only the 'participants' column. I will merge these two columns by converting the 'participants' column to match the ranges included in the 'participants_category' column. Some data will be lost in this process, as the 'participants category contains freeform text, not all of which is easily interpreted and converted to an actual figure. With more time, I will take a closer look at the rows that contain freeform text to convert to a numerical range. 
* protester_identity: I will drop this column for the current study, though will consider using it in future phases of the project. 
* protesterdemands / stateresponse: nulls will be replaced with 'unknown'
* sources and notes will be dropped

In [8]:
mm.isna().sum()

id                           0
country                      0
ccode                        0
year                         0
region                       0
protest                      0
protestnumber                0
startday                  1906
startmonth                1906
startyear                 1906
endday                    1906
endmonth                  1906
endyear                   1906
protesterviolence         1387
location                  1927
participants_category     7258
participants              1399
protesteridentity         2461
protesterdemand1          1907
protesterdemand2         14168
protesterdemand3         16762
protesterdemand4         16314
stateresponse1            1937
stateresponse2           14257
stateresponse3           16215
stateresponse4           16901
stateresponse5           16296
stateresponse6           17129
stateresponse7           16225
sources                   1910
notes                     1952
dtype: int64

In [9]:
# fill nulls in participants category with 'NaN' and setting up a formula for the replacement of null values with content from the participants column.

def f(df):
    if df['participants_category'] == 'NaN':
        return df['participants']
    else:
        return df['participants_category']

mm['participants_category'].fillna('NaN', inplace=True)

In [10]:
mm.shape

(17145, 31)

In [11]:
def clean_mass_mob_df(df=mm):
    
    #Drop nulls in startday/month/year and endday/month/year:
    df = df.drop(df[df["startyear"].isnull()].index)
    df.reset_index(inplace=True)

    
    #set startyear/month/day and endday/month/year to int, then string, in preparation for concatenation
    df = df.astype({"startyear": int, "startmonth": int, 'startday': int, "endyear": int, "endmonth": int, 'endday': int})
    df = df.astype({"startyear": str, "startmonth": str, 'startday': str, "endyear": str, "endmonth": str, 'endday': str})

    #create start date and end date columns and set as datetime object
    df['start_date'] = df['startyear'] + '-' + df['startmonth'] + '-'+ df['startday']
    df['end_date'] = df['endyear'] + '-' + df['endmonth'] + '-'+ df['endday']
    df['start_date'] = pd.to_datetime(df['start_date'], yearfirst=True)
    df['end_date'] = pd.to_datetime(df['end_date'], yearfirst=True)

    #setting df to only include dates between 2006 and 2020, in order to align with wgi and fiw data.
    df = df[(df['start_date']>'2006') & (df['start_date']<'2021')]

    #engineer a column to give duration of protest
    df['duration'] = df['end_date'] - df['start_date'] + timedelta(days=1)
    #reset duration column as an int
    df['duration_int'] = df['duration'].dt.days
    
    # converting protester violence from float to int, and filling nulls with 0
    df['protesterviolence'].fillna(0.0, inplace=True)
    df['protesterviolence'] = df['protesterviolence'].astype(int)
    
    
    #merge participants and participants category
    map_list = []
    for row in df['participants']:
        try:    
            map_list.append(row.strip('s><+ abcdefghijklmnopqrstuvwxyz!@#$%^&*():";."').split('-', 1)[0])
        except:
            map_list.append(row)
     
    df['participants'] = map_list
    #casting participants as numeric
    pd.to_numeric(df['participants'], errors='coerce').fillna(0, inplace=True)

    # setting participants to fit into participants_category ranges
    cat_map_list = []
    for row in df['participants']:
        try:
            if int(row) > 1 and int(row) < 100:
                cat_map_list.append('50-99')
            elif int(row) > 99 and int(row) < 1000:
                cat_map_list.append('100-999')
            elif int(row) > 999 and int(row) < 2000:
                cat_map_list.append('1000-1999')
            elif int(row) > 2000 and int(row) < 5000:
                cat_map_list.append('2000-4999')
            elif int(row) > 4999 and int(row) < 9999:
                cat_map_list.append('5000-10000')
            elif int(row) > 10000:
                cat_map_list.append('>10000')
            else:
                cat_map_list.append('unknown')
        except: 
            cat_map_list.append('unknown')

    df['participants'] = cat_map_list
      
    # mapping participants_category ranges to numerical category
    participants_category_map = {
        'unknown': 1,
        '50-99': 2,
        '100-999': 3,
        '1000-1999': 4,
        '2000-4999': 5,
        '5000-10000': 6,
        '>10000': 7
    }
    
    df['participants_category'].fillna('NaN', inplace=True)
    df['participants_category'] = df.apply(f, axis=1)
    df['participants_category'] = df['participants_category'].map(participants_category_map)
    #fill nulls with 'unknown'
    df.fillna('unknown', inplace=True)

    #resetting index
    df.reset_index(drop=True,inplace=True)

    #replacing country names in mm to align with wgi country names

    country_changes_mm = {'Macedonia': 'North Macedonia',
                          'Bosnia':'Bosnia and Herzegovina',
                          'Yugoslavia':'Serbia',
                          'Serbia and Montenegro': 'Montenegro',
                          'Russia': 'Russian Federation',
                          'Cape Verde': 'Cabo Verde',
                          'Gambia': 'Gambia, The',
                          'Ivory Coast': 'Cote d\'Ivoire',
                          'Congo Brazzaville': 'Congo, Rep.',
                          'Congo Kinshasa':'Congo, Dem. Rep.',
                          'Swaziland': 'Eswatini',
                          'United Arab Emirate': 'United Arab Emirates',
                          'Timor Leste':'Timor-Leste'}

    
    for name in df['country']:
        for k, v in country_changes_mm.items():
            if name == k:
                df.replace({name: v}, inplace=True)
  

    
    #renaming country column to country_name to align with wgi dataframe for merging
    df = df.rename({'country':'country_name'}, axis=1)
    
    # binarizing protester demand columns
    demands_df = df[['protesterdemand1', 'protesterdemand2', 'protesterdemand3', 'protesterdemand4']] 
    demands_df = demands_df.stack().str.get_dummies().sum(level=0).drop(columns=['.', 'unknown'])
    df = pd.concat([df, demands_df], axis=1)
 
    #drop original date columns
    df.drop(columns = ['startday', 'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
                       'index', 'location', 'sources', 'notes', 'participants',
                      'protesteridentity', 'protesterdemand1', 'protesterdemand2', 'protesterdemand3',
                       'protesterdemand4', 'stateresponse2', 'stateresponse3', 'stateresponse4',
                       'stateresponse5', 'stateresponse6', 'stateresponse7', 'id'], inplace=True)
    
    return df
    

In [12]:
mm = clean_mass_mob_df(mm)

In [13]:
mm.dtypes

country_name                            object
ccode                                    int64
year                                     int64
region                                  object
protest                                  int64
protestnumber                            int64
protesterviolence                        int64
participants_category                    int64
stateresponse1                          object
start_date                      datetime64[ns]
end_date                        datetime64[ns]
duration                       timedelta64[ns]
duration_int                             int64
labor wage dispute                       int64
land farm issue                          int64
police brutality                         int64
political behavior, process              int64
price increases, tax policy              int64
removal of politician                    int64
social restrictions                      int64
dtype: object

In [14]:
mm.shape

(8438, 20)

In [15]:
mm.isna().sum()

country_name                   0
ccode                          0
year                           0
region                         0
protest                        0
protestnumber                  0
protesterviolence              0
participants_category          0
stateresponse1                 0
start_date                     0
end_date                       0
duration                       0
duration_int                   0
labor wage dispute             0
land farm issue                0
police brutality               0
political behavior, process    0
price increases, tax policy    0
removal of politician          0
social restrictions            0
dtype: int64

In [16]:
#saving cleaned mass mobilization data as a csv
mm.to_csv('../data/transformed/mass_mobilization_data_cleaned.csv', index=False)

In [17]:
# checking for 60 countries with most appearances in mass mobilization dataset in order to select particular countries upon which to focus

mm_country_vals = pd.DataFrame(mm['country_name'].value_counts()[:60])['country_name'].index
mm_country_vals

Index(['Kenya', 'United Kingdom', 'Greece', 'Germany', 'Ireland', 'France',
       'China', 'Bangladesh', 'Thailand', 'India', 'Nigeria', 'Venezuela',
       'Yemen', 'Russian Federation', 'Namibia', 'Brazil', 'Kyrgyzstan',
       'Spain', 'Ukraine', 'Pakistan', 'Cyprus', 'Egypt', 'Turkey', 'Cuba',
       'Guinea', 'Lebanon', 'Iraq', 'Malaysia', 'Nepal', 'Italy', 'Chile',
       'Algeria', 'South Korea', 'Iran', 'Romania', 'Mexico', 'Madagascar',
       'Peru', 'Myanmar', 'Haiti', 'Burundi', 'South Africa', 'Sudan',
       'Honduras', 'Taiwan', 'Bolivia', 'Eswatini', 'Indonesia', 'Armenia',
       'Hungary', 'Zimbabwe', 'Colombia', 'Guatemala', 'Burkina Faso',
       'Mauritania', 'Togo', 'Moldova', 'North Macedonia', 'Ghana',
       'Congo, Dem. Rep.'],
      dtype='object')

### <a name="clean_fiw"></a>Cleaning Freedom in the World Data

Most columns will be dropped. I will retain only Country/Territory, Status, and Edition(year). 

[Jump to the top of this notebook](#top)

In [18]:
fiw.head()

,Country/Territory,Region,C/T?,Edition,Status,PR Rating,CL Rating,A,B,C,Add Q,Add A,PR,D,E,F,G,CL,Total,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,x
0,Abkhazia,Eurasia,t,2021,PF,5,5,5,8,4,0,NaN,17,8,6,4,5,23,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,Asia,c,2021,NF,5,6,3,7,3,0,NaN,13,6,4,2,2,14,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,Europe,c,2021,PF,3,3,8,12,7,0,NaN,27,13,8,9,9,39,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,MENA,c,2021,NF,6,5,3,4,3,0,NaN,10,6,3,6,7,22,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,Europe,c,2021,F,1,1,12,15,11,0,NaN,38,14,11,15,15,55,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
fiw.drop(columns=['Country/Territory', 'Status']).columns

Index(['Region', 'C/T?', 'Edition', 'PR Rating', 'CL Rating', 'A', 'B', 'C',
       'Add Q', 'Add A', 'PR', 'D', 'E', 'F', 'G', 'CL', 'Total',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'x'],
      dtype='object')

In [20]:
fiw.isna().sum()

Country/Territory       0
Region                  0
C/T?                    0
Edition                 0
Status                  0
PR Rating               0
CL Rating               0
A                       0
B                       0
C                       0
Add Q                   0
Add A                 838
PR                      0
D                       0
E                       0
F                       0
G                       0
CL                      0
Total                   0
Unnamed: 19          3339
Unnamed: 20          3339
Unnamed: 21          3339
Unnamed: 22          3339
Unnamed: 23          3339
Unnamed: 24          3339
Unnamed: 25          3339
Unnamed: 26          3339
Unnamed: 27          3339
Unnamed: 28          3339
Unnamed: 29          3339
Unnamed: 30          3339
Unnamed: 31          3339
Unnamed: 32          3339
Unnamed: 33          3339
Unnamed: 34          3339
x                    3339
dtype: int64

In [21]:
def clean_fiw_for_model(df=fiw):
    
    #preparing to drop all but my selected columns
    columns_to_drop = df.drop(columns=['Country/Territory', 'Status', 'Edition']).columns
    
    df.drop(columns=columns_to_drop, inplace=True)
    
    #renaming columns to match mm and wgi datasets to prepare for merging
    fiw_column_dict = {}
    for column in df.columns:
        if column == 'Country/Territory':
            fiw_column_dict[column] = 'country_name'
        elif column == 'Edition':
            fiw_column_dict[column] = 'year'
        else:
            fiw_column_dict[column] = 'fiw_' + column.lower().replace(' ', '_')

    df.rename(mapper=fiw_column_dict, axis=1, inplace=True)


    #replacing country names in mm to align with wgi country names

    country_changes_fiw = {'Slovakia': 'Slovak Republic',
                          'Serbia and Montenegro':'Serbia',
                          'Russia': 'Russian Federation',
                          'The Gambia': 'Gambia, The',
                          'Congo (Brazzaville)': 'Congo, Rep.',
                          'Congo (Kinshasa)':'Congo, Dem. Rep.' 
                          
}

    for name in df['country_name']:
        for k, v in country_changes_fiw.items():
            if name == k:
                df.replace({name: v}, inplace=True)

    #setting year column to a datetime object
    df['year'] = pd.to_datetime(df['year'], format='%Y', utc=True).dt.year
    
    #save cleaned df as a csv
    df.to_csv('../data/transformed/fiw_clean.csv', index=False)
    return df

In [22]:
fiw_clean = clean_fiw_for_model(fiw)

In [23]:
fiw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3339 entries, 0 to 3338
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country_name  3339 non-null   object
 1   year          3339 non-null   int64 
 2   fiw_status    3339 non-null   object
dtypes: int64(1), object(2)
memory usage: 78.4+ KB


In [24]:
#checking for countries' freedom categorization in 2006 and 2021.

fiw_2006_not_free_list = list(fiw_clean[(fiw_clean['year'] == 2006) & (fiw_clean['fiw_status'] == 'NF')]['country_name'])
fiw_2006_partially_free_list = list(fiw_clean[(fiw_clean['year'] == 2006) & (fiw_clean['fiw_status'] == 'PF')]['country_name'])
fiw_2006_free_list = list(fiw_clean[(fiw_clean['year'] == 2006) & (fiw_clean['fiw_status'] == 'F')]['country_name'])
fiw_2021_not_free_list = list(fiw_clean[(fiw_clean['year'] == 2021) & (fiw_clean['fiw_status'] == 'NF')]['country_name'])
fiw_2021_partially_free_list = list(fiw_clean[(fiw_clean['year'] == 2021) & (fiw_clean['fiw_status'] == 'PF')]['country_name'])
fiw_2021_free_list = list(fiw_clean[(fiw_clean['year'] == 2021) & (fiw_clean['fiw_status'] == 'F')]['country_name'])


In [25]:
#check for which countries in the various Freedom Scales categories by year occur most frequently in the Mass Mobilization data. 
nf_2006_mm_frequent = []
for country in fiw_2006_not_free_list:
    if country in mm_country_vals:
        nf_2006_mm_frequent.append(country)

print(nf_2006_mm_frequent)


['Algeria', 'China', 'Congo, Dem. Rep.', 'Cuba', 'Egypt', 'Guinea', 'Haiti', 'Iran', 'Iraq', 'Myanmar', 'Nepal', 'Pakistan', 'Russian Federation', 'Sudan', 'Togo', 'Zimbabwe']




Algeria, China, Cuba, Egypt, Guinea, Haiti, Iran, Iraq, Myanmar, Nepal, Pakistan, Russia, Sudan, Swaziland, Togo, and Zimbabwe intersect as appearing frequently in the mass mobilization dataset and were given the category of 'Not Free' in 2006. 



In [26]:
# A function to check the freedom score of a country in 2021.

def get_freedom_status_2006_2021(country_list):
    for country in country_list:
        if country in fiw_2021_free_list:
            print(f'{country} status is "Free" in 2021')
        if country in fiw_2021_partially_free_list:
            print(f'{country} status is "Partially Free" in 2021')
        if country in fiw_2021_not_free_list:
            print(f'{country} status is "Not Free" in 2021')


In [27]:
#countries designated as 'not free' in 2006. 

get_freedom_status_2006_2021(nf_2006_mm_frequent)

Algeria status is "Not Free" in 2021
China status is "Not Free" in 2021
Congo, Dem. Rep. status is "Not Free" in 2021
Cuba status is "Not Free" in 2021
Egypt status is "Not Free" in 2021
Guinea status is "Partially Free" in 2021
Haiti status is "Partially Free" in 2021
Iran status is "Not Free" in 2021
Iraq status is "Not Free" in 2021
Myanmar status is "Not Free" in 2021
Nepal status is "Partially Free" in 2021
Pakistan status is "Partially Free" in 2021
Russian Federation status is "Not Free" in 2021
Sudan status is "Not Free" in 2021
Togo status is "Partially Free" in 2021
Zimbabwe status is "Not Free" in 2021


In checking freedom status in 2021 for countries that were categorized as 'not free' in 2006, it doesn't appear that the needle has move significantly for many of these countries, though we do see a shift to 'partially free' for Guinea, Haiti, Nepal, Togo, and Pakistan.


In [28]:
#checking for most strongly represented countries categorized as partially free in 2006. 

pf_2006_mm_frequent = []
for country in fiw_2006_partially_free_list:
    if country in mm_country_vals:
        pf_2006_mm_frequent.append(country)

print(pf_2006_mm_frequent)

['Armenia', 'Bangladesh', 'Bolivia', 'Burkina Faso', 'Burundi', 'Colombia', 'Guatemala', 'Honduras', 'Kenya', 'Kyrgyzstan', 'Lebanon', 'Madagascar', 'Malaysia', 'Mauritania', 'Moldova', 'Nigeria', 'Thailand', 'Turkey', 'Venezuela', 'Yemen']


In [29]:
get_freedom_status_2006_2021(pf_2006_mm_frequent)

Armenia status is "Partially Free" in 2021
Bangladesh status is "Partially Free" in 2021
Bolivia status is "Partially Free" in 2021
Burkina Faso status is "Partially Free" in 2021
Burundi status is "Not Free" in 2021
Colombia status is "Partially Free" in 2021
Guatemala status is "Partially Free" in 2021
Honduras status is "Partially Free" in 2021
Kenya status is "Partially Free" in 2021
Kyrgyzstan status is "Not Free" in 2021
Lebanon status is "Partially Free" in 2021
Madagascar status is "Partially Free" in 2021
Malaysia status is "Partially Free" in 2021
Mauritania status is "Partially Free" in 2021
Moldova status is "Partially Free" in 2021
Nigeria status is "Partially Free" in 2021
Thailand status is "Not Free" in 2021
Turkey status is "Not Free" in 2021
Venezuela status is "Not Free" in 2021
Yemen status is "Not Free" in 2021


When checking 2021 status for those countries categorized as partially free in 2006, we do not see a single country that has now been categorized as free. The bulk of the countries remain categorized as partially free, with a few now categorized as not free in 2021. 

In [30]:
#checking for countries categorized as free in 2006 that appear most often in the mm dataset. 

f_2006_mm_frequent = []
for country in fiw_2006_free_list:
    if country in mm_country_vals:
        f_2006_mm_frequent.append(country)

print(f_2006_mm_frequent)

['Brazil', 'Chile', 'Cyprus', 'France', 'Germany', 'Ghana', 'Greece', 'Hungary', 'India', 'Indonesia', 'Ireland', 'Italy', 'Mexico', 'Namibia', 'Peru', 'Romania', 'South Africa', 'South Korea', 'Spain', 'Taiwan', 'Ukraine', 'United Kingdom']


In [31]:
get_freedom_status_2006_2021(f_2006_mm_frequent)

Brazil status is "Free" in 2021
Chile status is "Free" in 2021
Cyprus status is "Free" in 2021
France status is "Free" in 2021
Germany status is "Free" in 2021
Ghana status is "Free" in 2021
Greece status is "Free" in 2021
Hungary status is "Partially Free" in 2021
India status is "Partially Free" in 2021
Indonesia status is "Partially Free" in 2021
Ireland status is "Free" in 2021
Italy status is "Free" in 2021
Mexico status is "Partially Free" in 2021
Namibia status is "Free" in 2021
Peru status is "Partially Free" in 2021
Romania status is "Free" in 2021
South Africa status is "Free" in 2021
South Korea status is "Free" in 2021
Spain status is "Free" in 2021
Taiwan status is "Free" in 2021
Ukraine status is "Partially Free" in 2021
United Kingdom status is "Free" in 2021


A few countries that were categorized as free in 2006 were recategorized as partially free by 2021, but the bulk remain categorized as free. 

In [32]:
# Looking for which countries have seen variation in freedom status from 2006-2021

# I referenced stackoverflow for guidance on re-typing dictionary values

status_dict = {}
for i in fiw['country_name'].unique():
   status_dict[i] = [
         fiw['fiw_status'][j]
      for j in fiw[fiw['country_name'] == i].index
   ]

for keys in status_dict:
    status_dict[keys] = set(status_dict[keys])
    

In [33]:
for k, v in status_dict.items():
    if k in nf_2006_mm_frequent:
        print(k, v)

Algeria {'NF'}
China {'NF'}
Congo, Dem. Rep. {'NF'}
Cuba {'NF'}
Egypt {'NF', 'PF'}
Guinea {'NF', 'PF'}
Haiti {'NF', 'PF'}
Iran {'NF'}
Iraq {'NF'}
Myanmar {'NF', 'PF'}
Nepal {'NF', 'PF'}
Pakistan {'NF', 'PF'}
Russian Federation {'NF'}
Sudan {'NF'}
Togo {'NF', 'PF'}
Zimbabwe {'NF', 'PF'}


In [34]:
for k, v in status_dict.items():
    if k in pf_2006_mm_frequent:
        print(k, v)

Armenia {'PF'}
Bangladesh {'PF'}
Bolivia {'PF'}
Burkina Faso {'PF'}
Burundi {'NF', 'PF'}
Colombia {'PF'}
Guatemala {'PF'}
Honduras {'PF'}
Kenya {'PF'}
Kyrgyzstan {'NF', 'PF'}
Lebanon {'PF'}
Madagascar {'PF'}
Malaysia {'PF'}
Mauritania {'NF', 'PF'}
Moldova {'PF'}
Nigeria {'PF'}
Thailand {'NF', 'PF'}
Turkey {'NF', 'PF'}
Venezuela {'NF', 'PF'}
Yemen {'NF', 'PF'}


In [35]:
for k, v in status_dict.items():
    if k in f_2006_mm_frequent:
        print(k, v)

Brazil {'F'}
Chile {'F'}
Cyprus {'F'}
France {'F'}
Germany {'F'}
Ghana {'F'}
Greece {'F'}
Hungary {'F', 'PF'}
India {'F', 'PF'}
Indonesia {'F', 'PF'}
Ireland {'F'}
Italy {'F'}
Mexico {'F', 'PF'}
Namibia {'F'}
Peru {'F', 'PF'}
Romania {'F'}
South Africa {'F'}
South Korea {'F'}
Spain {'F'}
Taiwan {'F'}
Ukraine {'F', 'PF'}
United Kingdom {'F'}


In [36]:
for k, v in status_dict.items():
    if len(v) == 3:
        print(k, v)

Mali {'NF', 'F', 'PF'}
Tunisia {'NF', 'F', 'PF'}


In checking for countries that may have seen the most movement along the Freedom in the World scale, only 2 countries appear to have visited all three ranks (not free, partially free, and free) since 2006: Mali and Tunisia. 

### <a name="clean_wb"></a>Cleaning World Governance Indicators data

World Governance Indicators I will use during this study and modeling:
* Control of Corruption: Estimate (CC.EST)
    * Control of Corruption captures perceptions of the extent to which public power is exercised for private gain
* Government Effectiveness: Estimate (GE.EST)
    * Government Effectiveness captures perceptions of the quality of public service
* Political Stability and Absence of Violence/Terrorism: Estimate (PV.EST)
    * Political Stability and Absence of Violence/Terrorism measures perceptions of the likelihood of political instability and/or politically-motivated violence
* Rule of Law: Estimate
    * Rule of Law captures perceptions of the extent to which agents have confidence in and abide by the rules of society
* Voice and Accountability: Estimate
    * Voice and Accountability captures perceptions of the extent to which a country's citizens are able to participate in selecting their government
    
[Jump to the top of this notebook](#top)

In [37]:
wgi.head(3)

,Country Name,Country Code,Series Name,Series Code,1996 [YR1996],1998 [YR1998],2000 [YR2000],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Afghanistan,AFG,Control of Corruption: Estimate,CC.EST,-1.291705,-1.180848,-1.29538,-1.263366,-1.351042,-1.345281,-1.443609,-1.43289,-1.587331,-1.638287,-1.534796,-1.636177,-1.579174,-1.419741,-1.43651,-1.354829,-1.342216,-1.526172,-1.515626,-1.487624,-1.400733,-1.475405
1,Afghanistan,AFG,Control of Corruption: Number of Sources,CC.NO.SRC,2,2,2,2,3,5,5,7,6,7,8,9,9,10,11,11,11,10,10,10,10,9
2,Afghanistan,AFG,Control of Corruption: Percentile Rank,CC.PER.RNK,4.301075,9.793815,5.076142,5.050505,5.050505,5.853659,2.439024,3.902439,0.9708738,0.4854369,0.9569378,0.952381,0.9478673,2.369668,1.895735,5.288462,6.25,3.365385,3.846154,4.807693,6.730769,5.288462


In [38]:
def clean_databank(df, csv_name):
    
    #dropping country code column
    df.drop(columns=['Country Code', 'Series Name'], inplace=True)
    
    #rename year columns
    for column in df.columns[2:]:
        df.rename(columns = {column: column[:4]}, inplace=True)
        
    for column in df.columns[:2]:
        df.rename(columns = {column: column.lower().replace(' ', '_')}, inplace=True)
        
    #create columns for 1997, 1999, and 2001, which will replicate the previous year, so that I have data for each year for analysis purposes. 
    #However, I will only use 2006-2021 data in modeling, for use with the mm and fiw data. 
    df['1997'] = df['1996']
    df['1999'] = df['1998']
    df['2001'] = df['2000']
    
    #reorder columns
    df = df[['country_name', 'series_code', '1996', '1997', '1998', '1999', '2000',
       '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020']]
    
    #dropping unneeded series codes
    db_series = ['CC.EST', 'GE.EST', 'PV.NO.SRC', 'RL.EST', 'VA.EST', 'SI.POV.GINI', 'SP.POP.TOTL', 'SI.SPR.BL50.ZS', 'SI.POV.ATTM.MI',
                 'SI.POV.ENRL.MI', 'SI.POV.WATR.MI', 'SI.POV.ELEC.MI', 'SI.DST.05TH.20', 'NY.GDP.PCAP.CD', 'NY.GDP.PCAP.KD.ZG']
    df = df[df['series_code'].isin(db_series)]
    
    #replacing country names in wgi to align with mm
    country_changes_db = {'Venezuela, RB': 'Venezuela',
                          'Iran, Islamic Rep.': 'Iran',
                          'Egypt, Arab Rep.': 'Egypt',
                          'Syrian Arab Republic': 'Syria',
                          'Yemen, Rep.': 'Yemen',
                          'Kyrgyz Republic': 'Kyrgyzstan',
                          'Taiwan, China': 'Taiwan',
                          'Korea, Dem. People\'s Rep.': 'North Korea',
                          'Korea, Rep.': 'South Korea'}

    for name in df['country_name']:
        for k, v in country_changes_db.items():
            if name == k:
                df.replace({name: v}, inplace=True)
    
    df.reset_index(drop=True, inplace=True)
    
    # using melt to get rows and country_name set for merging with other dataframes
    # referenced stackoverflow for melt documentation: https://stackoverflow.com/questions/28654047/convert-columns-into-rows-with-pandas
    df = df.melt(id_vars=['country_name', 'series_code'])
    df.rename(columns = {'variable': 'year', 'value': 'score'}, inplace=True)
    
    # setting year as datetime object
    df['year'] = pd.to_datetime(df['year'], format='%Y').dt.year
    
    #converting to a pivot table to get the format needed to merge with mm. 
    df_pivot = df.pivot(index=['country_name','year'], columns=['series_code'], values=['score'])
    df = pd.DataFrame(df_pivot)
    
    #filling fields containing '..' with NaNs.
    df.replace('..',np.NaN, inplace=True)
    
    
    #I save this cleaned data as a csv, then open the csv in excel to reset the headers, before reading the data back in to merge with mm and fiw. 
    # df.to_csv(f'../data/transformed/{csv_name}_pivot.csv')

#     return df
    


In [39]:
# clean_databank(wgi, 'wgi')

In [40]:
#in order to convert this pivot table to a dataframe, I manually edited out headers in excel, and read in the formatted csv..

wgi_df = pd.read_csv('../data/transformed/wgi_pivot.csv')



In [41]:
wgi_df.head()

,CC.EST,GE.EST,PV.NO.SRC,RL.EST,VA.EST,country_name,year
0,-1.291705,-2.175167,2.0,-1.788075,-1.908540,Afghanistan,1996
1,-1.291705,-2.175167,2.0,-1.788075,-1.908540,Afghanistan,1997
2,-1.180848,-2.135942,2.0,-1.737140,-2.039301,Afghanistan,1998
3,-1.180848,-2.135942,2.0,-1.737140,-2.039301,Afghanistan,1999
4,-1.295380,-2.231651,2.0,-1.778437,-2.031417,Afghanistan,2000


In [42]:
wgi_df.shape

(5564, 7)

In [43]:
wgi_2021 = wgi_df[wgi_df['year'] == 2020]

In [44]:
wgi_2021['year'] = 2021

<ipython-input-44-4795ffcf0192>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wgi_2021['year'] = 2021


In [46]:
# WGI scores are currently unavailable for 2021. I am copying the scores from 2020 as fillers until that data is available. 
# wgi_reset = wgi_df.append(wgi_2021, ignore_index=True, verify_integrity=False, sort=True)
wgi_reset.head(3)

,CC.EST,GE.EST,PV.NO.SRC,RL.EST,VA.EST,country_name,year
0,-1.291705,-2.175167,2.0,-1.788075,-1.908540,Afghanistan,1996
1,-1.291705,-2.175167,2.0,-1.788075,-1.908540,Afghanistan,1997
2,-1.180848,-2.135942,2.0,-1.737140,-2.039301,Afghanistan,1998


In [47]:
wgi_reset[wgi_reset['year'] == 2021]

,CC.EST,GE.EST,PV.NO.SRC,RL.EST,VA.EST,country_name,year
5350,-1.475405,-1.523115,5.0,-1.807697,-1.083459,Afghanistan,2021
5351,-0.540196,-0.135743,6.0,-0.358294,0.087340,Albania,2021
5352,-0.641845,-0.525844,5.0,-0.780318,-1.102139,Algeria,2021
5353,1.314374,0.696851,2.0,1.193629,1.056903,American Samoa,2021
5354,1.314374,1.825594,3.0,1.698638,1.089068,Andorra,2021
...,...,...,...,...,...,...,...
5773,0.043223,0.696851,2.0,0.941124,NaN,Virgin Islands (U.S.),2021
5774,-0.550139,-0.643822,4.0,-0.461406,-0.826022,West Bank and Gaza,2021
5775,-1.679567,-2.307661,5.0,-1.776561,-1.767067,Yemen,2021
5776,-0.704854,-0.767911,6.0,-0.622515,-0.430640,Zambia,2021


In [48]:
# saving reset wgi csv

# wgi_reset.to_csv('../data/transformed/wgi_pivot.csv', index=False)

In [49]:
wgi_reset.isna().sum()

CC.EST          250
GE.EST          270
PV.NO.SRC       244
RL.EST          168
VA.EST          197
country_name      0
year              0
dtype: int64

#### <a name="merge"></a> Merge and Export the Data

[Jump to the top of this notebook](#top)

In [50]:
mm.shape

(8438, 20)

In [51]:
#merging wgi and mm dataframes

mm_wgi = mm.merge(wgi_reset, how='inner', left_on=["country_name", "year"], right_on=["country_name","year"])

In [52]:
#checking for nulls in merged data
mm_wgi.isna().sum()

country_name                   0
ccode                          0
year                           0
region                         0
protest                        0
protestnumber                  0
protesterviolence              0
participants_category          0
stateresponse1                 0
start_date                     0
end_date                       0
duration                       0
duration_int                   0
labor wage dispute             0
land farm issue                0
police brutality               0
political behavior, process    0
price increases, tax policy    0
removal of politician          0
social restrictions            0
CC.EST                         0
GE.EST                         0
PV.NO.SRC                      0
RL.EST                         0
VA.EST                         0
dtype: int64

In [53]:
# saving merged mm and wgi dataframe to csv
mm_wgi.to_csv('../data/transformed/mm_wgi.csv', index=False)

In [54]:
# checking to ensure that no countries will be lost when merging with Freedom in the World data. 
merged_df_countries = list(mm_wgi['country_name'].unique())
fiw_countries = list(fiw_clean['country_name'].unique())

for country in merged_df_countries:
    if country not in fiw_countries:
        print(country)

In [55]:
mm_wgi.shape

(8438, 25)

In [56]:
# merging fiw with mm_wgi dataframe

mm_wgi_fiw = mm_wgi.merge(fiw_clean, how='inner', left_on=['country_name', 'year'], right_on=["country_name", 'year'])

In [57]:
mm_wgi_fiw.shape

(8338, 26)

In [58]:
# renaming countries to align with those need to use with plotly, based on cross comparison with comprehensive list from datahub: https://datahub.io/core/country-codes

cc = pd.read_csv('../data/raw/country-codes_csv.csv')

#updated country names to match format needed for plotly

plotly_countries = cc['UNTERM English Short'].unique()

df_countries = mm_wgi_fiw['country_name'].unique()

for country in df_countries:
    if country not in plotly_countries:
        print(country)

Dominican Republic
Venezuela
Bolivia
United Kingdom
Netherlands
Czech Republic
Slovak Republic
Kosovo
North Macedonia
Moldova
Russian Federation
Gambia, The
Niger
Cote d'Ivoire
Central African Republic
Congo, Rep.
Congo, Dem. Rep.
Tanzania
Comoros
Sudan
Iran
Syria
United Arab Emirates
Taiwan
North Korea
South Korea
Vietnam
Philippines


In [59]:
country_map_plotly = {'Dominican Republic': 'Dominican Republic (the)',
'Venezuela': 'Venezuela (Bolivarian Republic of)',
'Bolivia': 'Bolivia (Plurinational State of)',
'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland (the)',
'Netherlands': 'Netherlands (the)', 
'Czech Republic': 'Czech Republic (the)',
'Slovak Republic': 'Slovakia',
'North Macedonia': 'the former Yugoslav Republic of Macedonia', 
'Moldova': 'Republic of Moldova (the)',
'Russian Federation': 'Russian Federation (the)',
'Gambia, The': 'Gambia (the)',
'Niger': 'Niger (the)',
"Cote d'Ivoire": "Côte d'Ivoire",
'Central African Republic': 'Central African Republic (the)',
'Congo, Rep.': 'Congo (the)',
'Congo, Dem. Rep.': 'Democratic Republic of the Congo (the)',
'Tanzania':'United Republic of Tanzania (the)',
'Comoros': 'Comoros (the)', 
'Sudan': 'Sudan (the)',
'Iran': 'Iran (Islamic Republic of)', 
'Syria': 'Syrian Arab Republic', 
'United Arab Emirates': 'United Arab Emirates (the)',
'North Korea': "Democratic People's Republic of Korea (the)",
'South Korea': 'Republic of Korea (the)',
'Vietnam': 'Viet Nam',
'Philippines': 'Philippines (the)'}


In [60]:
for country in mm_wgi_fiw['country_name']:
    for k, v in country_map_plotly.items():
        if country == k:
            mm_wgi_fiw.replace({country: v}, inplace=True)

In [61]:
mm_wgi_fiw.head(3)

,country_name,ccode,year,region,protest,protestnumber,protesterviolence,participants_category,stateresponse1,start_date,end_date,duration,duration_int,labor wage dispute,land farm issue,police brutality,"political behavior, process","price increases, tax policy",removal of politician,social restrictions,CC.EST,GE.EST,PV.NO.SRC,RL.EST,VA.EST,fiw_status
0,Canada,20,2006,North America,1,1,1,3,crowd dispersal,2006-02-21,2006-08-17,178 days,178,0,0,1,1,0,0,0,1.962554,1.895545,8.0,1.814535,1.415595,F
1,Canada,20,2007,North America,1,1,0,4,ignore,2007-06-29,2007-06-29,1 days,1,0,0,0,1,0,0,0,2.000900,1.761394,8.0,1.814706,1.384672,F
2,Canada,20,2008,North America,1,1,1,3,crowd dispersal,2008-08-10,2008-08-10,1 days,1,0,0,1,0,0,0,0,2.002689,1.791681,8.0,1.808290,1.407529,F


In [62]:
plotly_countries = cc['UNTERM English Short'].unique()

df_countries = mm_wgi_fiw['country_name'].unique()

for country in df_countries:
    if country not in plotly_countries:
        print(country)

Kosovo
Taiwan


In [63]:
#saving final merged mm - wgi - fiw dataframe to csv

mm_wgi_fiw.to_csv('../data/transformed/mm_wgi_fiw.csv', index=False)

[Jump to the top of this notebook](#top)